In [25]:
using DrWatson
using ITensors, ITensorMPS
import ITensors.hasinds
using HDF5

@quickactivate "hoping_chain_tensors"

[ Info: Precompiling ITensorMPS [0d1a4710-d33b-49a5-8f18-73bdf49b47e2]


# DMRG example

From https://itensor.github.io/ITensors.jl/dev/#Singular-Value-Decomposition-(SVD)-of-a-Matrix for reference.


In [27]:
let
    # Create 100 spin-one indices
    N = 100
    # makes an array of 100 site-indices which have the properties of S=1 spins (i.e three values -1,0,1)
    sites = siteinds("S=1",N) # Tag is "S=1"

    # Here we will define the parts of the hamiltonian in a bit of an abstract way
    os = OpSum() # Accumulates hamiltonian terms so they can be later summed  into an MPO 
    # iterate over sites
    for j=1:N-1
        os += "Sz",j,"Sz",j+1
        os += 0.5,"S+",j,"S-",j+1
        os += 0.5,"S-",j,"S+",j+1
    end
    H = MPO(os,sites) # construct hamiltonian in MPO format

    # Create an initial random matrix product state
    # the initial wavefunction guess has indices "sites" and bond dimension (linkdims) = 10
    # random_mps means that it is constructed from a MPS random quantum circuit.
    # Helps avoid getting stuck in local minima
    psi0 = random_mps(sites, linkdims=10)

    # Do 10 DMR sweeps
    nsweeps = 5
    # Set the maximum bond dimension for each of the 5 sweeps
    maxdim = [10,20,100,100,200]
    # Truncation error goal. Can be one for each sweep
    # or just the same for all of them
    cutoff = 1E-10

    # Run the DMRG algorithm, returning energy
    # (dominant eigenvalue) and optimized MPS
    energy, psi = dmrg(H,psi0; nsweeps, maxdim, cutoff)
    # After the dmrg function returns, you can take the returned MPS psi and 
    # do further calculations with it, such as measuring local operators or 
    # computing entanglement entropy
    println("Final energy = $energy")

    nothing
end


After sweep 1 energy=-138.83629108206335  maxlinkdim=10 maxerr=1.51E-02 time=0.101
After sweep 2 energy=-138.93731849509896  maxlinkdim=20 maxerr=4.71E-06 time=0.198
After sweep 3 energy=-138.94008430176675  maxlinkdim=90 maxerr=1.00E-10 time=1.173
After sweep 4 energy=-138.940086054326  maxlinkdim=99 maxerr=9.99E-11 time=3.573
After sweep 5 energy=-138.94008605321278  maxlinkdim=95 maxerr=9.98E-11 time=3.067
Final energy = -138.94008605321278


# Thight Binding Periodic Boundaries